In [1]:
import re
import traceback
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, length, concat_ws, regexp_replace, size, split
from pyspark.sql.types import StringType, IntegerType, DoubleType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer,
    VectorAssembler, NGram, Word2Vec
)
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# --- Helper UDF (Python UDFs will always run on CPU. Cannot be GPU-accelerated directly.) ---
def clean_tags(tags):
    if tags is None: return []
    return [tag.strip() for tag in tags.replace('<', ' ').replace('>', ' ').split() if tag.strip()]

clean_tags_udf = udf(clean_tags, ArrayType(StringType()))

# --- Spark Session Configuration ---
# Re-emphasize: Ensure only one compatible cudf JAR is in your classpath.
# The 'allowMultipleJars' config is a workaround, not a solution for conflicting JARs.
spark = (SparkSession.builder
    .appName("GPU_IMP_RECOMMENDED")
    .master("local[*]")
    .config("spark.driver.memory","24g") # Keep increased memory
    .config("spark.executor.memory", "24g") # Keep increased memory
    .config("spark.executor.cores", "4")
    .config("spark.driver.cores", "4")
    .config("spark.sql.shuffle.partitions", "200")
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin")
    .config("spark.driver.host", "localhost") # Keep for now, but if issues persist, try removing or setting to system IP
    .config("spark.rapids.sql.explain", "ALL") # Retain for detailed logging
    .config("spark.rapids.sql.allowMultipleJars", "ALWAYS") # Still a workaround, resolve actual JAR duplicates if possible
    .config("spark.rapids.sql.enabled", "true")
    .config("spark.rapids.memory.hostStorageFraction", "0.8")
    .config("spark.rapids.memory.deviceStorageFraction", "0.8")
    .config("spark.rapids.sql.concurrentGpuTasks", "2")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size", "8g") # Keep increased off-heap memory
    .config("spark.rapids.memory.gpu.allocation.limit", "0.9")
    .config("spark.rapids.host.shim.async", "true")
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.rapids.sql.exec.CollectLimitExec", "true") 
    .config("spark.rapids.sql.rowBasedUDF.enabled", "true") # Essential for CPU fallback of unsupported UDFs
    # Crucial for GPU-backed in-memory caching of Spark SQL DataFrames
    .config("spark.sql.inMemoryColumnarStorage.batchSerializer", "com.nvidia.spark.rapids.shims.SparkShimServiceProvider")
    .getOrCreate()
)

data_path = "/mnt/c/Users/BerenÜnveren/Desktop/BIL401/data/train.csv"

try:
    print("--- Data Loading ---")
    start_load_time = time.time()
    # CSV loading with custom escape characters is likely CPU-bound, as noted previously.
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("multiLine", "true") \
        .option("inferSchema", "true") \
        .load(data_path)
    load_time = time.time() - start_load_time
    print(f"Data load time: {load_time:.2f} seconds")

    print("data schema:")
    df.printSchema()
    print("Y column distribution after load (should be HQ, LQ_EDIT, LQ_CLOSE):")
    df.groupBy("Y").count().show()

    print("--- Data Cleaning and Feature Engineering (initial steps on CPU) ---")
    start_clean_feature_time = time.time()
    df_clean = df.na.drop(subset=["Title", "Body", "Tags", "Y"]) \
        .withColumn("CleanBody", regexp_replace(col("Body"), "<.*?>", "")) \
        .withColumn("text", concat_ws(" ", col("Title"), col("CleanBody"))) \
        .withColumn("tags_list", clean_tags_udf(col("Tags"))) # This Python UDF runs on CPU.

    df_featured = df_clean.withColumn("title_len", length(col("Title"))) \
        .withColumn("body_len", length(col("CleanBody"))) \
        .withColumn("punct_count", length(regexp_replace(col("text"), "[?!]", ""))) \
        .withColumn("avg_word_len", length(regexp_replace(col("text"), " ", "")) / (size(split(col("text"), " ")) + 1e-6))
    clean_feature_time = time.time() - start_clean_feature_time
    print(f"Initial data cleaning and feature engineering time: {clean_feature_time:.2f} seconds")

    # Define MLlib feature transformers. These will largely run on CPU due to VectorUDT.
    label_indexer = StringIndexer(inputCol="Y", outputCol="label", handleInvalid="skip")
    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
    ngram = NGram(n=2, inputCol="filtered_words", outputCol="bigrams")
    hashing_tf_text = HashingTF(inputCol="filtered_words", outputCol="raw_text_features", numFeatures=20000)
    idf_text = IDF(inputCol="raw_text_features", outputCol="text_features")
    hashing_tf_bigrams = HashingTF(inputCol="bigrams", outputCol="raw_bigrams_features", numFeatures=20000)
    idf_bigrams = IDF(inputCol="raw_bigrams_features", outputCol="bigrams_features")
    w2v = Word2Vec(vectorSize=100, minCount=5, inputCol="filtered_words", outputCol="w2v_features")
    hashing_tf_tags = HashingTF(inputCol="tags_list", outputCol="raw_tags_features", numFeatures=5000)
    idf_tags = IDF(inputCol="raw_tags_features", outputCol="tags_tags")
    feature_assembler = VectorAssembler(
        inputCols=["text_features", "bigrams_features", "w2v_features", "tags_tags",
                   "title_len", "body_len", "punct_count", "avg_word_len"],
        outputCol="features"
    )

    # --- Create a dedicated feature engineering pipeline and run it once ---
    # This pipeline will run on CPU, but we run it once and cache the result.
    feature_pipeline = Pipeline(stages=[
        label_indexer, tokenizer, stopwords_remover, ngram,
        hashing_tf_text, idf_text, hashing_tf_bigrams, idf_bigrams, w2v, hashing_tf_tags, idf_tags,
        feature_assembler
    ])

    print("\n--- Fitting Feature Engineering Pipeline ---")
    start_feature_fit_time = time.time()
    feature_model = feature_pipeline.fit(df_featured)
    end_feature_fit_time = time.time()
    print(f"Feature engineering pipeline fit time: {end_feature_fit_time - start_feature_fit_time:.2f} seconds")

    print("\n--- Transforming data with Feature Engineering Pipeline ---")
    start_feature_transform_time = time.time()
    df_transformed = feature_model.transform(df_featured)
    end_feature_transform_time = time.time()
    print(f"Feature engineering transformation time: {end_feature_transform_time - start_feature_transform_time:.2f} seconds")

    # Split the transformed data
    (train_data_final, test_data_final) = df_transformed.randomSplit([0.8, 0.2], seed=42)

    # Cache the final feature-engineered DataFrames.
    # With spark.sql.inMemoryColumnarStorage.batchSerializer set,
    # Spark will attempt to cache these in GPU memory.
    print("\n--- Caching final train and test data ---")
    train_data_final.cache()
    test_data_final.cache()
    
    # Trigger an action to force the caching to happen immediately
    train_data_final.count()
    test_data_final.count()

    print(f"Train data count (after feature engineering): {train_data_final.count()}, Test data count (after feature engineering): {test_data_final.count()}")

    # --- Logistic Regression Training ---
    print("\n--- Logistic Regression Training ---")
    # Now, the Logistic Regression model itself should be able to leverage GPU if RAPIDS supports its solver.
    # The pipeline is simpler, just the estimator, as features are pre-computed.
    lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

    start_lr_train_time = time.time()
    lr_model = lr.fit(train_data_final) # Train directly on the prepared data
    lr_train_time = time.time() - start_lr_train_time
    print(f"Logistic Regression training time: {lr_train_time:.2f} seconds")

    start_lr_predict_time = time.time()
    lr_predictions = lr_model.transform(test_data_final)
    lr_predict_time = time.time() - start_lr_predict_time
    print(f"Logistic Regression prediction time: {lr_predict_time:.2f} seconds")

    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
    accuracy_lr = evaluator.setMetricName("accuracy").evaluate(lr_predictions)
    f1_score_lr = evaluator.setMetricName("f1").evaluate(lr_predictions)

    print("\nLogistic Regression Results:")
    print(f"Accuracy: {accuracy_lr:.4f}")
    print(f"F1 Score: {f1_score_lr:.4f}")
    print("Confusion Matrix:")
    lr_predictions.groupBy("label", "prediction").count().orderBy("label", "prediction").show()

    # --- Random Forest Training with Cross-Validation ---
    print("\n--- Random Forest Training with Cross-Validation ---")
    # Similarly for Random Forest, train directly on the prepared features.
    # The CrossValidator will now repeatedly fit only the RandomForestClassifier, not the entire feature pipeline.
    rf = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42)

    paramGrid = ParamGridBuilder() \
        .addGrid(rf.numTrees, [50, 100]) \
        .addGrid(rf.maxDepth, [5, 10]) \
        .build()

    # Create a new CrossValidator that uses just the RandomForestClassifier on the pre-processed data
    crossval_rf = CrossValidator(estimator=rf,
                                 estimatorParamMaps=paramGrid,
                                 evaluator=MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1"),
                                 numFolds=3)

    start_rf_train_time = time.time()
    cv_model = crossval_rf.fit(train_data_final) # Train directly on the prepared data
    rf_train_time = time.time() - start_rf_train_time
    print(f"Random Forest training (with CV) time: {rf_train_time:.2f} seconds")

    best_rf_model = cv_model.bestModel
    start_rf_predict_time = time.time()
    rf_predictions = best_rf_model.transform(test_data_final)
    rf_predict_time = time.time() - start_rf_predict_time
    print(f"Random Forest prediction time: {rf_predict_time:.2f} seconds")

    accuracy_rf = evaluator.setMetricName("accuracy").evaluate(rf_predictions)
    f1_score_rf = evaluator.setMetricName("f1").evaluate(rf_predictions)

    print("\nRandom Forest Results:")
    print(f"Accuracy: {accuracy_rf:.4f}")
    print(f"F1 Score: {f1_score_rf:.4f}")

    best_params_stage = best_rf_model # The best_rf_model itself is the RandomForestClassifier model
    print(f"Best parameters: numTrees={best_params_stage.getNumTrees()}, maxDepth={best_params_stage.getMaxDepth()}")

    print("Confusion Matrix:")
    rf_predictions.groupBy("label", "prediction").count().orderBy("label", "prediction").show()

except Exception as e:
    print(f"An error occurred: {e}")
    traceback.print_exc()
finally:
    try:
        spark.stop()
    except Exception as e:
        print(f"Error stopping Spark session: {e}")
        traceback.print_exc()

25/07/22 23:18:23 WARN Utils: Your hostname, DESKTOP-15VE119 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/22 23:18:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/22 23:18:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/22 23:18:24 WARN RapidsPluginUtils: RAPIDS Accelerator 24.02.0 using cudf 24.02.1.
25/07/22 23:18:24 WARN RapidsPluginUtils: spark.rapids.sql.multiThreadedRead.numThreads is set to 20.
25/07/22 23:18:24 WARN RapidsPluginUtils: Multiple cudf jars found in the classpath:
revison: dd34fdbe35e68ba56a2183f11ed822ddaa6c927b
	jar URL: jar:file:/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/pyspark/jars/rapids-4-spark_2.12-24.02.0.jar
	version=24.02.1
	user=
	

--- Data Loading ---


25/07/22 23:18:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

Data load time: 5.02 seconds
data schema:
root
 |-- Id: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Y: string (nullable = true)

Y column distribution after load (should be HQ, LQ_EDIT, LQ_CLOSE):


25/07/22 23:18:44 WARN GpuOverrides: 
*Exec <CollectLimitExec> will run on GPU
  *Partitioning <SinglePartition$> will run on GPU
  *Exec <HashAggregateExec> will run on GPU
    *Expression <AggregateExpression> count(1) will run on GPU
      *Expression <Count> count(1) will run on GPU
    *Expression <Alias> toprettystring(Y#5, Some(UTC)) AS toprettystring(Y)#25 will run on GPU
      *Expression <ToPrettyString> toprettystring(Y#5, Some(UTC)) will run on GPU
    *Expression <Alias> toprettystring(count(1)#18L, Some(UTC)) AS toprettystring(count)#26 will run on GPU
      *Expression <ToPrettyString> toprettystring(count(1)#18L, Some(UTC)) will run on GPU
    *Exec <ShuffleExchangeExec> will run on GPU
      *Partitioning <HashPartitioning> will run on GPU
      *Exec <HashAggregateExec> will run on GPU
        *Expression <AggregateExpression> partial_count(1) will run on GPU
          *Expression <Count> count(1) will run on GPU
        !Exec <FileSourceScanExec> cannot run on GPU be

...

25/07/22 23:22:19 WARN GpuOverrides: 
!Exec <FilterExec> cannot run on GPU because unsupported data types in input: org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 [raw_text_features#564, w2v_features#663, text_features#586, raw_bigrams_features#611, raw_tags_features#691, tags_tags#718, bigrams_features#635, features#750]; unsupported data types in output: org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 [raw_text_features#564, w2v_features#663, text_features#586, raw_bigrams_features#611, raw_tags_features#691, tags_tags#718, bigrams_features#635, features#750]
  @Expression <And> ((CrossValidator_969240a92c92_rand#7259 >= 0.0) AND (CrossValidator_969240a92c92_rand#7259 < 0.3333333333333333)) could run on GPU
    @Expression <GreaterThanOrEqual> (CrossValidator_969240a92c92_rand#7259 >= 0.0) could run on GPU
      @Expression <AttributeReference> CrossValidator_969240a92c92_rand#7259 could run on GPU
      @Expression <Literal> 0.0 could run on GPU
    @Expression <LessThan> (CrossValidat

An error occurred: An error occurred while calling o856.evaluate


Traceback (most recent call last):
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/bunvere

Error stopping Spark session: [Errno 111] Connection refused


Traceback (most recent call last):
  File "/tmp/ipykernel_7471/298080422.py", line 215, in <module>
    spark.stop()
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/pyspark/sql/session.py", line 1799, in stop
    self._jvm.SparkSession.clearDefaultSession()
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/py4j/java_gateway.py", line 1712, in __getattr__
    answer = self._gateway_client.send_command(
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/py4j/java_gateway.py", line 1036, in send_command
    connection = self._get_connection()
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/py4j/clientserver.py", line 284, in _get_connection
    connection = self._create_new_connection()
  File "/home/bunveren/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/py4j/clientserver.py", line 291, in _create_new_connection
    connection.connect_to_java_server()
